In [ ]:
# https://www.postgresqltutorial.com/postgresql-tutorial/postgresql-group-by/

# GROUP BY : - Divides by the row returned by select statement into groups.
#              For each group you can apply aggregate function like COUNT, SUM, MIN, MAX, etc.
#            - Eliminate duplicate values from the result, same as DISTINC.


# syntax: 
    
SELECT 
   column_1, 
   column_2,
   ...,
   aggregate_function(column_3)
FROM 
   table_name
GROUP BY 
   column_1,
   column_2,


# Example_1

SELECT count(movie_lang) FROM movies 

count|
-----+
   53|

# Can also using another agregate function    
SELECT movie_lang, count(movie_lang) FROM movies 
GROUP BY movie_lang

movie_lang|count|
----------+-----+
Portuguese|    2|
German    |    1|
Chinese   |    5|
English   |   38|
Swedish   |    1|
Spanish   |    1|
Korean    |    1|
Japanese  |    4|


SELECT movie_lang, MIN(movie_lang), MAX(movie_lang) FROM movies 
GROUP BY movie_lang 

movie_lang|min       |max       |
----------+----------+----------+
Portuguese|Portuguese|Portuguese|
German    |German    |German    |
Chinese   |Chinese   |Chinese   |
English   |English   |English   |
Swedish   |Swedish   |Swedish   |
Spanish   |Spanish   |Spanish   |
Korean    |Korean    |Korean    |
Japanese  |Japanese  |Japanese  |



In [ ]:
# Using multiple column with group by

 
# on group : there must have movie_lang, age_certificate if SELECT clause have 2 columns
# you will get an error if group only one column
SELECT movie_lang, age_certificate, AVG(movie_length)
FROM movies 
GROUP BY movie_lang, age_certificate  

SELECT * FROM movies

movie_lang|age_certificate|avg                 |
----------+---------------+--------------------+
Japanese  |18             |109.5000000000000000|
Chinese   |15             |113.0000000000000000|
English   |18             |125.0000000000000000|
Spanish   |PG             | 98.0000000000000000|
Swedish   |15             |128.0000000000000000|
Chinese   |12             |124.0000000000000000|


# Using where and order clasue : attention about position
SELECT movie_lang, age_certificate, AVG(movie_length)
FROM movies 
WHERE movie_length > 100
GROUP BY movie_lang, age_certificate, movie_length 
ORDER BY movie_length 

movie_lang|age_certificate|avg                 |
----------+---------------+--------------------+
English   |12             |104.0000000000000000|
Japanese  |U              |107.0000000000000000|
Japanese  |18             |108.0000000000000000|
Japanese  |18             |111.0000000000000000|
English   |PG             |112.0000000000000000|


# Using SUM
SELECT movie_lang, age_certificate, SUM(movie_length)
FROM movies 
GROUP BY movie_lang, age_certificate 
ORDER BY 3 DESC 

movie_lang|age_certificate|sum |
----------+---------------+----+
English   |15             |1638|
English   |PG             |1364|
English   |12             | 929|
English   |18             | 500|
Chinese   |12             | 496|
English   |U              | 393|






In [ ]:
# Order of executions in GROUP BY

# FROM 

# WHERE --condition

# GROUP BY --group sets

# HAVING --filter again

# SELECT --columns

# DISTINC --unique columns if you use DISTINC

# ORDER BY 

# LIMIT --filter records



In [ ]:
# HAVING 
# - Specify search conditions for a group or an aggregate
# - The having clause is often used  GROUP by clause to filter row based or to filter conditions
# - You cannot column aliases in the HAVING clause, as HAVING is evaluated before SELECT clause
# - Having clause calculate on aggregate functions and not actual columns

# Intinya HAVING itu kerjaanya filter hasil dari aggregate (works on result group)


syntax :
    
SELECT 
   column_1, 
   column_2,
   ...,
   aggregate_function(column_3)
FROM 
   table_name
GROUP BY 
   column_1,
   column_2,
HAVINGs
   conditions aggregate_function(column_3)

    
# Example_1
# Wanto to know about movie languge how many translation
SELECT movie_lang, SUM(movie_length)
FROM movies 
GROUP BY movie_lang 
HAVING sum(movie_length) > 200

movie_lang|sum |
----------+----+
Portuguese| 285|
Chinese   | 609|
English   |4824|
Japanese  | 446|


# Want to know director id and his movies production
SELECT director_id, SUM(movie_length)
FROM movies
GROUP BY director_id 
HAVING SUM(movie_length) > 200
ORDER BY director_id 

director_id|sum|
-----------+---+
          3|340|
          5|272|
          8|243|
         13|368|


In [ ]:
# Order of executions of the HAVING

# FROM 

# WHERE --condition

# GROUP BY --group sets

# HAVING --filter again

# SELECT --columns

# DISTINC --unique columns if you use DISTINC

# ORDER BY 

# LIMIT --filter records



In [ ]:
# Handling NUll value in Group By
# USING COALESCE
# - COALESCE(source, '') <-- source: source of the data, '': replace of the data
# - COALESCE(departement, 'NO Department')

# New table
CREATE TABLE employees_test(
	employee_id SERIAL PRIMARY KEY, 
	employee_name VARCHAR(100),
	department VARCHAR(100),
	salary INT
)

# Insert it mockup
INSERT INTO employees_test (employee_name, department, salary) VALUES
('Jhon', 'Finance', 2500),
('Mary', NULL, 3000),
('Adam', NULL, 4000),
('Bruce', 'Finance', 4000),
('Lynda', 'IT', 5000)


# when we select all 
SELECT * FROM employees_test

# There's a NULL
employee_id|employee_name|department|salary|
-----------+-------------+----------+------+
          1|Jhon         |Finance   |  2500|
          2|Mary         |          |  3000|
          3|Adam         |          |  4000|
          4|Bruce        |Finance   |  4000|
          5|Lynda        |IT        |  5000|
            
# Use CAOLESCE to replace source            
SELECT COALESCE(department, 'No Department') AS Department, COUNT(salary) AS total_employees
FROM employees_test 
GROUP BY department 

# Boom
department   |total_employees|
-------------+---------------+
No Department|              2|
Finance      |              2|
IT           |              1|

In [ ]:
#  ORDER BY clause and NULL

# When you sort rows that contains NULL, you can specify the order of NULL with other non-null values 
# by using the NULLS FIRST or NULLS LAST option of the ORDER BY clause:

# The NULLS FIRST option places NULL before other non-null values and 
# the NULL LAST option places NULL after other non-null values.

# Intinya klo misalnya ada null, taroh di atas atau dibawah null nya

ORDER BY sort_expresssion [ASC | DESC] [NULLS FIRST | NULLS LAST]


SELECT 
	a.movie_name, 
	b.first_name, 
	b.last_name,
	c.revenues_domestic, 
	c.revenues_international,
	(c.revenues_domestic + c.revenues_international) AS "Total Revenues"
FROM movies a
INNER JOIN directors b ON a.movie_id  = b.director_id 
INNER JOIN movies_revenues c ON a.movie_id = c.revenue_id
ORDER BY 6 DESC NULLS LAST 
LIMIT 5

movie_name                    |first_name|last_name|revenues_domestic|revenues_international|Total Revenues|
------------------------------+----------+---------+-----------------+----------------------+--------------+
Never Let Me Go               |Sam       |Raimi    |           659.20|               1528.10|       2187.30|
Spider-Man 3                  |Quentin   |Tarantino|           404.10|                418.10|        822.20|
Star Wars: Empire Strikes Back|Robert    |Zemeckis |           374.10|                410.40|        784.50|
Chungking Express             |Luc       |Besson   |           461.20|                314.20|        775.40|
Life of Brian                 |Hayao     |Miyazaki |           330.30|                348.10|        678.40|